In [1]:
## This script is to plot mean DA MUCAPE, mean NoDA MUCAPE, and mean RAOB MUCAPE
## from Tucson (TUS) and Flagstaff (FGZ) at 00 UTC and 12 UTC >>>>>>>> STEP 1
## go to next step ....
## Created by C. Bayu Risanto, S.J. (9 September 2025)
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from numpy import matlib
import pandas as pd
import xarray as xr
from scipy import stats
from scipy.io import loadmat,savemat 
from scipy.interpolate import griddata
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.colors as mplotcolors
from matplotlib import cm
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from datetime import date, datetime, timedelta
import glob
def read_nx(ncfile):
    dx = xr.open_dataset(ncfile)
    mcape = dx.mcape[:,:]
    return mcape
def read_latlon(ncfile):
    dx = xr.open_dataset(ncfile)
    lat = dx.XLAT[0,:,:]
    lon = dx.XLONG[0,:,:]
    return lat,lon
##### Define the bilinear interpolation function #######################################
def bilinear_interpolation(x, y, lon, lat, data):
    points = np.column_stack((lon.values.ravel(), lat.values.ravel()))
    values = data.ravel()
    interp_values = griddata(points, values, (x, y), method='linear')
    return interp_values

In [2]:
## days of rain
DR = ['2021-07-03','2021-07-09','2021-07-10','2021-07-12','2021-07-13','2021-07-14',
      '2021-07-15','2021-07-16','2021-07-18','2021-07-21','2021-07-22','2021-07-23',
      '2021-07-24','2021-07-25','2021-07-29','2021-07-30','2021-08-10','2021-08-11',
      '2021-08-13','2021-08-15','2021-08-16','2021-08-30','2021-08-31','2021-09-01']

In [3]:
## read data DETER both DA and NODA
path = '/secchi/bayu/DATA/ARIZONA_PRO/WRF_EXTR/WRF_OUT'
hor = [18,19,20,21,22,23,00,1,2,3,4,5,6,7,8,9,10,11,12]
aCY = [] ; aNY = []
for ih in hor:
    mcapCY = [] ; mcapNY = []
    for i in range(len(DR)):
        dir1 = DR[i][0:4]+DR[i][5:7]+DR[i][8:10]+'_06Z_mc08_40mem_0035_6cyc_assim'
        dir2 = DR[i][0:4]+DR[i][5:7]+DR[i][8:10]+'_06Z_mc08_40mem_0035_6cyc_noassim'
        fCY = 'ncl-2D_d02_*_'+str(ih).zfill(2)+':00:00.nc'
        fNY = 'ncl-2D_d02_*_'+str(ih).zfill(2)+':00:00.nc'
        Lcy = glob.glob(path+'/'+dir1+'/'+fCY); Lny = glob.glob(path+'/'+dir2+'/'+fNY)
        capCY = read_nx(Lcy[0])
        capNY = read_nx(Lny[0])
        mcapCY.append(capCY); mcapNY.append(capNY)
    mcapCY = np.asarray(mcapCY); mcapNY = np.asarray(mcapNY)
    aCY.append(mcapCY); aNY.append(mcapNY)
aCY = np.asarray(aCY); aNY = np.asarray(aNY)

In [4]:
## Calculate the mean
mu_CY = np.nanmean(aCY,axis=1)
mu_NY = np.nanmean(aNY,axis=1)

In [5]:
print(mu_NY.shape)

(19, 405, 684)


In [6]:
## get latlon
pathlx = '/secchi/bayu/DATA/ARIZONA_PRO/metadata/'
xlat,xlon = read_latlon(pathlx+'lonlat.nc')

In [7]:
## EXTRACT Tucson and Flagstaff 
lat_TUS = 32.25
lon_TUS = -110.97 

lat_FGZ = 35.1983
lon_FGZ = -111.6513

DAMU_tus = []; NOMU_tus = [] ; DAMU_fgz = []; NOMU_fgz = []
for i in range(len(mu_CY[:,1,1])):
    DAmucape_tus = bilinear_interpolation(lon_TUS,lat_TUS,xlon,xlat,mu_CY[i,:,:])
    DAmucape_fgz = bilinear_interpolation(lon_FGZ,lat_FGZ,xlon,xlat,mu_CY[i,:,:])

    NOmucape_tus = bilinear_interpolation(lon_TUS,lat_TUS,xlon,xlat,mu_NY[i,:,:])
    NOmucape_fgz = bilinear_interpolation(lon_FGZ,lat_FGZ,xlon,xlat,mu_NY[i,:,:])

    DAMU_tus.append(DAmucape_tus); NOMU_tus.append(NOmucape_tus);
    DAMU_fgz.append(DAmucape_fgz); NOMU_fgz.append(NOmucape_fgz);
DAMU_tus = np.asarray(DAMU_tus); NOMU_tus = np.asarray(NOMU_tus);
DAMU_fgz = np.asarray(DAMU_fgz); NOMU_fgz = np.asarray(NOMU_fgz)


In [10]:
print(DAMU_tus)
print(NOMU_tus)

[1075.72866162 1340.90816564 1318.74787352 1196.05855836 1215.52951094
 1173.99736793  942.22350992  799.4817218   661.93880341  628.97571064
  603.45959412  560.56466729  467.14537204  364.05756111  340.25650313
  338.44130141  284.87070485  287.85810001  277.24634663]
[332.89135417 381.33550002 425.44395675 417.12367078 449.82987577
 454.85762312 433.18028587 354.29769002 382.32215349 422.80191055
 639.34536369 535.13168635 471.6993718  480.19781499 404.05695529
 415.49744211 419.759565   403.90667151 409.35427509]


In [9]:
## save the results here to avoid a long time calculation
dir_out = '/secchi/bayu/DATA/ARIZONA_PRO/WRF_EXTR/WRF-RAOB/MUCAPE_extr/'
fileout = 'TUS_mean_MUCAPE_DANoDA.mat' 
output_file0 = dir_out + fileout
mdic0 = {'DA':DAMU_tus,'NoDA':NOMU_tus}
savemat(output_file0,mdic0)

fileout = 'FGZ_mean_MUCAPE_DANoDA.mat' 
output_file0 = dir_out + fileout
mdic0 = {'DA':DAMU_fgz,'NoDA':NOMU_fgz}
savemat(output_file0,mdic0)